In [1]:
from datetime import datetime
import yaml
from backbone.utils.general_purpose import load_function
from backtest.utils import plot_full_equity_curve, walk_forward
import pandas as pd


root = './backbone/data'

with open('configs/live_trading.yml', 'r') as file:
    strategies = yaml.safe_load(file)

with open('configs/test_creds.yml', 'r') as file:
    creds = yaml.safe_load(file)

date_from = datetime(2020, 1, 1)
date_to = datetime(2024, 9, 1)

not_run = [
    'backbone.eom_trader.EndOfMonthTrader',
    'backbone.vix_trader.VixTrader',    
    'backbone.b_percent_trader.BPercentTrader',
    # 'backbone.macd_trader.MacdTrader',
    'backbone.mean_reversion_trader.MeanRevTrader',
    'backbone.bbands_cross_trader.BbandsCrossTrader',
    # 'backbone.day_per_week_trader.DayPerWeekTrader'
]

equity_curves = {}
trades = {}

for bot_name, configs in strategies.items():

    instruments_info = configs['instruments_info']
    wfo_params = configs['wfo_params']
    opt_params = configs['opt_params']

    if bot_name in not_run:
        continue

    for ticker, info in instruments_info.items():

        timeframe = info['timeframe']
        
        name = f'{bot_name.split(".")[-1]}_{ticker}_{timeframe}'
        print(name)
        
        cron = info['cron']
        timeframe = info['timeframe']
        contract_volume = info['contract_volume']
    
        bot = load_function(bot_name)(ticker, timeframe, contract_volume, creds, opt_params, wfo_params)
        
        if bot_name == 'backbone.vix_trader.VixTrader':
            df = bot.get_full_data(date_from, date_to)
 
        else:
            df = bot.trader.get_data(date_from, date_to)
        
        
        if ticker == 'US500m' or ticker == 'USTECm' or ticker == 'US30m':
            fracc_df = df * 0.01
        else:
            fracc_df = df


        wfo_stats = walk_forward(
            bot.strategy,
            fracc_df, 
            lookback_bars=bot.wfo_params['look_back_bars'],
            validation_bars=250,
            warmup_bars=bot.wfo_params['warmup_bars'], 
            params=bot.opt_params,
            commission=7e-4, 
            margin=1/30, 
            cash=10_000,
            verbose=True
        )
        
        equity_curves[name] = wfo_stats['_equity']
        trades[name] = wfo_stats['_trades']
        
        
        
    

MacdTrader_XPEVm_M15
  login=82847194
  trade_mode=0
  leverage=500
  limit_orders=1024
  margin_so_mode=0
  trade_allowed=True
  trade_expert=True
  margin_mode=2
  currency_digits=2
  fifo_close=False
  balance=1371.56
  credit=0.0
  profit=-2.16
  equity=1369.4
  margin=41.11
  margin_free=1328.29
  margin_level=3331.0630017027493
  margin_so_call=60.0
  margin_so_so=0.0
  margin_initial=0.0
  margin_maintenance=0.0
  assets=0.0
  liabilities=0.0
  commission_blocked=0.0
  name=Said
  server=Exness-MT5Trial11
  currency=USD
  company=Exness Technologies Ltd
train from 2021-10-28 17:00:00 to 2021-12-14 15:30:00
validate from 2021-12-14 15:45:00 to 2021-12-23 11:30:00
{'cum_rsi_open_threshold': 75, 'cum_rsi_close_threshold': 55, 'risk': 1}
{'cum_rsi_open_threshold': 75, 'cum_rsi_close_threshold': 55, 'risk': 1}
equity final: 10273.996744478623
train from 2021-11-08 13:00:00 to 2021-12-23 11:30:00
validate from 2021-12-23 12:00:00 to 2022-01-04 14:00:00
{'cum_rsi_open_threshold': 75, '

In [7]:
import pandas as pd

min_date = None
max_date = None

for name, curve in equity_curves.items():
    # Convertir las fechas a UTC si son tz-naive
    actual_date = curve.index[0].tz_localize('UTC') if curve.index[0].tz is None else curve.index[0].tz_convert('UTC')
    
    # Si min_date es None, inicializar con la primera fecha
    if min_date is None:
        min_date = actual_date
    # Comparar si la fecha actual es menor que min_date
    elif actual_date < min_date:
        min_date = actual_date

    # Si max_date es None, inicializar con la última fecha
    curve_last_date = curve.index[-1].tz_localize('UTC') if curve.index[-1].tz is None else curve.index[-1].tz_convert('UTC')
    
    if max_date is None:
        max_date = curve_last_date
    # Comparar si la fecha actual es mayor que max_date
    elif curve_last_date > max_date:
        max_date = curve_last_date

# Mostrar las fechas encontradas
print(f"Min Date: {min_date}")
print(f"Max Date: {max_date}")

# Calcular min_date y max_date
min_date = min_date.date()
max_date = max_date.date()

print(min_date)
print(max_date)


date_range = pd.to_datetime(pd.date_range(start=min_date, end=max_date, freq='15min'))
print(date_range)

Min Date: 2021-12-14 00:00:00+00:00
Max Date: 2024-08-29 16:15:00+00:00
2021-12-14
2024-08-29
DatetimeIndex(['2021-12-14 00:00:00', '2021-12-14 00:15:00',
               '2021-12-14 00:30:00', '2021-12-14 00:45:00',
               '2021-12-14 01:00:00', '2021-12-14 01:15:00',
               '2021-12-14 01:30:00', '2021-12-14 01:45:00',
               '2021-12-14 02:00:00', '2021-12-14 02:15:00',
               ...
               '2024-08-28 21:45:00', '2024-08-28 22:00:00',
               '2024-08-28 22:15:00', '2024-08-28 22:30:00',
               '2024-08-28 22:45:00', '2024-08-28 23:00:00',
               '2024-08-28 23:15:00', '2024-08-28 23:30:00',
               '2024-08-28 23:45:00', '2024-08-29 00:00:00'],
              dtype='datetime64[ns]', length=94945, freq='15T')


In [9]:
del equity_curves['all']

In [10]:
initial_value = 10_000

total = pd.DataFrame()

for name, curve in equity_curves.items():

    eq = equity_curves[name].copy()
    trad = trades[name][['ExitTime', 'PnL']].copy()

    # Renombro las columnas
    trad = trad.rename(columns={'ExitTime':'Date'})
    eq = eq.reset_index().rename(columns={'index':'Date'})[['Date','Equity']].sort_values(by='Date')

    trad['Date'] = pd.to_datetime(trad['Date'])
    eq['Date'] = pd.to_datetime(eq['Date'])

    # joineo los dfs por fechas
    full_df = pd.merge(
        eq,
        trad,
        on='Date',
        how='left'   
    )

    full_df = full_df[~full_df['PnL'].isna()]

    full_df['Date'] = full_df['Date'].dt.floor('D')

    full_df = full_df.groupby('Date').agg({'Equity':'last', 'PnL':'sum'})


    full_df.fillna(0, inplace=True)
    total = pd.concat([total, full_df])


total = total.sort_values(by='Date')


total = total.groupby(by='Date').agg({'Equity':'sum','PnL':'sum'}).reset_index().rename(columns={'index':'Date'})
total = total[total.PnL != 0].copy()


first_row = pd.DataFrame(
    {
        'Date':[pd.to_datetime('2021-12-14')],
        'Equity':[initial_value],
        'PnL':[0],
        'pct':[0],
    }
)

total = pd.concat(
    [first_row, total]
).reset_index().drop(columns=['index'])

total['pct'] = total['PnL'] / total['Equity'].shift(1)

total = total.fillna(0)

total


,Date,Equity,PnL,pct
0,2021-12-14,10000.000000,0.000000,0.000000
1,2021-12-15,9978.196581,-21.803419,-0.002180
2,2021-12-16,10041.438627,63.242046,0.006338
3,2021-12-17,10081.053728,39.615102,0.003945
4,2021-12-20,10273.996744,192.943016,0.019139
...,...,...,...,...
475,2024-08-14,20269.411998,791.023042,0.040610
476,2024-08-15,19994.832663,-274.579335,-0.013546
477,2024-08-20,19925.218779,-69.613884,-0.003482
478,2024-08-21,19715.422922,-209.795857,-0.010529


In [11]:
import numpy as np
import pandas as pd

# Serie de porcentajes de cambio
initial_equity = 10_000

# Calcular el comportamiento de la cartera aplicando los porcentajes de cambio
equity_curve = initial_equity * np.cumprod(1 + total.pct)

# Convertir en DataFrame
equity_curve = pd.DataFrame(equity_curve)
equity_curve['Date'] = total.Date

equity_curve = equity_curve.set_index('Date').rename(columns={'pct':'Equity'})

# equity_curve = equity_curve.reindex(date_range)
equity_curve['Equity'] = equity_curve['Equity'].ffill().fillna(initial_equity)

# Mostrar el resultado
equity_curve

,Equity
Date,
2021-12-14,10000.000000
2021-12-15,9978.196581
2021-12-16,10041.438627
2021-12-17,10081.053728
2021-12-20,10273.996744
...,...
2024-08-14,18772.057794
2024-08-15,18517.762349
2024-08-20,18453.291024


In [20]:
import plotly.graph_objects as go

equity_curves['simulated_equity_curve'] = equity_curve


# Crear una figura vacía
fig = go.Figure()

# Recorrer las curvas de equity de cada bot y agregarlas al gráfico
for k, v in equity_curves.items():
    
    fig.add_trace(go.Scatter(x=v.index, y=v.Equity, mode='lines', name=k))

# Actualizar los detalles del layout del gráfico
fig.update_layout(
    title="Curvas de Equity de Múltiples Bots",
    xaxis_title="Fecha",
    yaxis_title="Equity",
    legend_title="Bots"
)

# Mostrar el gráfico
fig.show()


In [17]:
equity_curves['DayPerWeekTrader_AMZNm_H4'].tail()

,Equity,DrawdownPct,DrawdownDuration
2024-08-06 16:00:00,9717.035888,0.025451,NaT
2024-08-07 08:00:00,9717.035888,0.025451,NaT
2024-08-07 12:00:00,9717.035888,0.025451,NaT
2024-08-07 16:00:00,9717.035888,0.025451,NaT
2024-08-08 08:00:00,9717.035888,0.025451,114 days


In [16]:
equity_curves['DayPerWeekTrader_TSMm_H4'].tail()

,Equity,DrawdownPct,DrawdownDuration
2024-08-06 16:00:00,10530.768613,0.035642,NaT
2024-08-07 08:00:00,10530.768613,0.035642,NaT
2024-08-07 12:00:00,10530.768613,0.035642,NaT
2024-08-07 16:00:00,10530.768613,0.035642,NaT
2024-08-08 08:00:00,10530.768613,0.035642,24 days


In [18]:
equity_curves['MacdTrader_XPEVm_M15'].tail()

,Equity,DrawdownPct,DrawdownDuration
2024-08-29 15:15:00,19321.89743,0.030279,NaT
2024-08-29 15:30:00,19321.89743,0.030279,NaT
2024-08-29 15:45:00,19321.89743,0.030279,NaT
2024-08-29 16:00:00,19321.89743,0.030279,NaT
2024-08-29 16:15:00,19321.89743,0.030279,8 days 02:00:00
